In [7]:
# Diagnost the boston dataset
import os
from src.models import vanilla_predNet, MC_dropnet, Deep_Ensemble
from src.losses import mse_loss, rmse_loss, mean_std_norm_loss, mean_std_forEnsemble, BeyondPinball_muSigma, MMD_Loss, MACE_Loss, MACE_muSigma
import torch
from src.GPmodels import oneLayer_DeepGP
import time
import copy
import numpy as np
import operator
import yaml
from Experiments.EXP1.trainer import trainer
from data_utils import seed_all, splitter, common_processor_UCI, get_uci_data



SEED = 1234
dataset_name = "boston"
dataname = "boston"
dataset_path = "Experiments/EXP1/UCI_datasets"

model_callByName = {
    "GPmodel": oneLayer_DeepGP,
    "HNN": MC_dropnet,
    "MC_drop": MC_dropnet,
    "DeepEnsemble": Deep_Ensemble,
    "HNN_BeyondPinball": MC_dropnet,
    "vanillaPred": vanilla_predNet,

    "HNN_MMD": MC_dropnet,
    "vanillaMSQR": vanilla_predNet,
    "vanillaKernel": vanilla_predNet
}

loss_callByName = {
    "mse_loss": mse_loss, 
    "rmse_loss": rmse_loss, 
    "mean_std_norm_loss": mean_std_norm_loss, 
    "mean_std_forEnsemble": mean_std_forEnsemble, 
    "BeyondPinball_muSigma": BeyondPinball_muSigma,
    "MMD_Loss": MMD_Loss,
    "MACE_Loss": MACE_Loss,
    "MACE_muSigma": MACE_muSigma
}




x, y = get_uci_data(data_name= dataset_name, dir_name= dataset_path)

train_X, test_X, recal_X, train_Y, test_Y, recal_Y = common_processor_UCI(x, y, recal_percent= 0.5, seed = SEED)

train_X = torch.Tensor(train_X)
train_Y = torch.Tensor(train_Y).to(torch.device("cuda"))



with open("Experiments/EXP1/config_bin/vanillaPred_on_"+dataname+"_config.yml", 'r') as file:
    base_configs = yaml.safe_load(file)

base_misc_info = base_configs["misc_info"]
base_train_config= base_configs["training_config"]


base_model = model_callByName[base_misc_info["model_init"]](**base_misc_info["model_config"])


trainer(
    seed = SEED,
    raw_train_X = train_X,
    raw_train_Y = train_Y,
    model = base_model,
    training_config = base_train_config,
    harvestor = None,          
    misc_info = base_misc_info,
    diff_trainingset = True
)


In [ ]:
"rmse_loss": rmse_loss, 
"mean_std_norm_loss": mean_std_norm_loss, 
    
    
"MACE_muSigma": MACE_muSigma,

"AGCE_muSigma": AGCE_muSigma,

"CheckScore_muSigma": avg_pinball_muSigma

In [15]:
import pandas as pd



dic = {
    "hh":1,
    "oo":2
}

In [17]:
list(dic.values())

[1, 2]

In [ ]:
recal_X = torch.Tensor(recal_X)
            recal_Y = torch.Tensor(recal_Y).to(torch.device(base_misc_info["model_config"]["device"]))

            # we need to get those val_x and val_y
            split_percet = base_misc_info["val_percentage"]
    
            N_val = int(split_percet*len(train_Y))

            train_idx, val_idx = splitter(len(train_Y)-N_val, N_val, seed = SEED)

            val_X, val_Y = train_X[val_idx], train_Y[val_idx]

            recal_mean = base_model(recal_X).view(-1)
            val_mean = base_model(val_X).view(-1)
            
            assert "wid" in to_search

            for wid in to_search["wid"]:

                eps_diffQuants = kernel_estimator(
                    test_Z = val_X.cuda(),
                    recal_Z = recal_X.cuda(),
                    recal_epsilon = torch.Tensor(recal_Y - recal_mean).cuda(),
                    quants = np.linspace(0.01,0.99,100),
                    wid= wid
                )

                y_diffQuants = eps_diffQuants + val_mean.view(1,-1).repeat(len(eps_diffQuants),1)
                MACE_error = MACE_Loss(y_diffQuants,val_Y,q_list = np.linspace(0.01,0.99,100)).item()

In [ ]:
aux_misc_info:
  input_x_shape:
  - 8611
  - 4
  input_y_shape:
  - 8611
  model_config:
    device: cuda
    drop_rate: 0.0
    hidden_layers:
    - 10
    - 5
    n_input: 4
    n_output: 2
  model_init: HNN_MMD
  save_path_and_name: null
  val_percentage: 0.1
aux_train_config:
  Decay: 0.0001
  LR: 0.001
  N_Epoch: 200
  backdoor: null
  bat_size: 128
  early_stopping: true
  monitor_name: MMD
  patience: 20
  train_loss: MMD_Loss
  val_loss_criterias:
    MACE: MACE_muSigma
    MMD: MMD_Loss
    nll: mean_std_norm_loss
    rmse: rmse_loss
  validate_times: 20
  verbose: false
base_misc_info:
  input_x_shape:
  - 8611
  - 4
  input_y_shape:
  - 8611
  model_config:
    device: cuda
    drop_rate: 0.0
    hidden_layers:
    - 10
    - 5
    n_input: 4
    n_output: 2
  model_init: HNN_MMD
  save_path_and_name: null
  val_percentage: 0.1
base_train_config:
  Decay: 0.0001
  LR: 0.005
  N_Epoch: 200
  backdoor: null
  bat_size: 64
  early_stopping: true
  monitor_name: nll
  patience: 20
  train_loss: mean_std_norm_loss
  val_loss_criterias:
    MACE: MACE_muSigma
    nll: mean_std_norm_loss
    rmse: rmse_loss
  validate_times: 20
  verbose: false
harvestor:
  early_stopped: false
  early_stopping_epoch: 0
  monitor_name: MACE
  monitor_vals: []
  training_losses: []
  val_MACE: []
  val_MMD: []
  val_nll: []
  val_rmse: []


In [ ]:

        if model_name == "HNN_MMD": 
            # base model will be reused

            aux_misc_info = copy.deepcopy(base_misc_info)
            aux_train_config = copy.deepcopy(base_train_config)

            aux_train_config["train_loss"] = "MMD_Loss"

            aux_train_config["val_loss_criterias"] = {
                "MMD": "MMD_Loss",
                "MACE": "MACE_muSigma",
                "nll": "mean_std_norm_loss",
                "rmse": "rmse_loss"
            }
            aux_train_config["monitor_name"] = "MMD"

            harvestor = {
                "early_stopped": False,
                "early_stopping_epoch": 0,
                "monitor_name": "MACE",
                "monitor_vals": [],
                "training_losses": [],
                "val_MMD": [],
                "val_MACE": [],
                "val_nll": [],
                "val_rmse": []
            }



            assert "LR" in to_search.keys() and "bat_size" in to_search.keys()

            for LR in to_search["LR"]:
                for bat_size in to_search["bat_size"]:
                    aux_train_config["LR"] = LR
                    aux_train_config["bat_size"] = bat_size

                    empty_harvestor(harvestor)

                    model_reuse = copy.deepcopy(base_model)

                    trainer(
                        seed = SEED,
                        raw_train_X = train_X,
                        raw_train_Y = train_Y,
                        model = model_reuse,
                        training_config = aux_train_config,
                        harvestor = harvestor,          
                        misc_info = aux_misc_info
                    )

                
                    sub_summarizer[(LR, bat_size)] = np.mean(harvestor["monitor_vals"][-3:])
                    aid_summarizer[(LR, bat_size)] = {}

                    for key in aux_train_config["val_loss_criterias"].keys():
                        aid_summarizer[(LR, bat_size)]["val_"+key] = np.mean(harvestor["val_"+key][-3:])


            para_got = min(sub_summarizer.items(), key=operator.itemgetter(1))[0]
            summarizer.append(para_got)
            support_summ[para_got] = aid_summarizer[para_got]

